# AIF360: Fairness in AI Toolkit

AIF360 es un conjunto de herramientas diseñadas para abordar la equidad y la justicia en los sistemas de inteligencia artificial (IA). Ofrece una serie de algoritmos, métricas y utilidades para ayudar a los desarrolladores y los profesionales de IA a comprender, medir y mitigar los sesgos y las disparidades en los modelos de IA.

## Utilización

- **Evaluar la equidad:** Proporciona métricas predefinidas y personalizables para evaluar la equidad de los modelos de IA en función de atributos sensibles como género, raza, edad, etc.
- **Mitigar sesgos:** Ofrece algoritmos y técnicas para mitigar los sesgos identificados en los modelos de IA, ya sea en el preprocesamiento de datos, durante el entrenamiento del modelo o en la etapa de postprocesamiento.
- **Comprender el impacto de las decisiones de IA:** Permite a los usuarios comprender cómo los modelos de IA pueden afectar a diferentes grupos demográficos y cómo esos impactos pueden ser injustos o discriminatorios.
- **Promover la equidad y la justicia:** Facilita el diseño de sistemas de IA más equitativos y justos al proporcionar herramientas para identificar y abordar los sesgos, lo que lleva a una toma de decisiones más inclusiva y ética.

#### Librerias requeridas

In [91]:
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing.reweighing import Reweighing
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from IPython.display import Markdown, display
from aif360.algorithms.preprocessing.lfr import LFR
import pandas as pd

#### Lectura del dataset

In [92]:
df = pd.read_csv('../data/Statlog_preprocesado.csv')

#### El AIF360 requiere de un dataset de tipo numérico para poder trabajar, por lo que se realiza una conversión de las variables categóricas a numéricas.

Se realiza un label encoder porque se ha comprobado que con one hot encoder el modelo pierde precisión en los pesos. Solamente se utilizará este tipo de encoding en este notebook para poder trabajar con el AIF360, posteriormente se volverá a utilizar el dataset original con los pesos obtenidos con AIF360.

In [93]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

data_encoded = df.copy(deep=True)

#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical_columns:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)

Feature Status of existing checking account
mapping {'0 - 200 DM': 0, '< 0 DM': 1, '>= 200 DM': 2, 'no checking account': 3}
Feature Credit history
mapping {'all credits paid': 0, 'critical account': 1, 'delay in paying': 2, 'existing credits paid': 3, 'no credits/all paid': 4}
Feature Purpose
mapping {'business': 0, 'car (new)': 1, 'car (used)': 2, 'domestic appliances': 3, 'education': 4, 'furniture/equipment': 5, 'others': 6, 'radio/television': 7, 'repairs': 8, 'retraining': 9}
Feature Savings account/bonds
mapping {'100 - 500 DM': 0, '500 - 1000 DM': 1, '< 100 DM': 2, '>= 1000 DM': 3, 'unknown/no savings': 4}
Feature Present employment since
mapping {'1 - 4 years': 0, '4 - 7 years': 1, '< 1 year': 2, '>= 7 years': 3, 'unemployed': 4}
Feature Other debtors / guarantors
mapping {'co-applicant': 0, 'guarantor': 1, 'none': 2}
Feature Property
mapping {'building society savings': 0, 'car or other': 1, 'real estate': 2, 'unknown/no property': 3}
Feature Other installment plans
mapping {

#### Se realiza un split del dataset en train y test

In [94]:
X = data_encoded
X = X.drop('Class', axis=1)
y = data_encoded['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### Se convierte el dataframe de Pandas en un objeto BinaryLabelDataset para poder trabajar con AIF360

Es necesario definir ciertos parámetros para poder realizar este tipo de conjunto de datos:

1. Asignación de variable objetivo y sus valores

In [95]:
target_label = ['Class']
favorable_labels = 1  # 'Concedido'
unfavorable_labels = 2   # 'Denegado'

2. Asignación de variables protegidas y sus valores

Estas son variables que pueden generar algún sesgo en la sociedad como la edad, el genero, la raza...

In [96]:
protected_attributes = [
    'Gender',
    'Foreign worker',
    'Marital Status',
    'Job'
]

privileged_groups = [
    {
        'Gender': 1, # Hombre
        'Foreign worker': 0, # 'no'
        'Marital Status': 2, # 'married'
        'Job': 0, # 'management/self-employed'
        'Job': 1, # 'skilled employee'
     }
]

unprivileged_groups = [
    {
        'Gender': 0,  # Mujer
        'Foreign worker': 1, # 'yes'
        'Marital Status': 0,  # 'divorced/separated'
        'Marital Status': 3,  # 'single'
        'Marital Status': 1,   # 'divorced/separated/married'
        'Job': 2,  # 'unemployed/non-resident'
        'Job': 3   # 'unskilled resident'
    }  
]


En este caso se eligen las siguientes variables como protegidas:
- Genero
- Trabajador extranjero (se entiende que es inmigrante)
- Estado civil
- Trabajo (Se entiende que tener _skills_ va directamente relacionado con el nivel de estudios)

In [97]:
binary_dataset = BinaryLabelDataset(df=data_encoded,
                                      label_names=target_label,
                                      favorable_label=favorable_labels,
                                      unfavorable_label=unfavorable_labels,
                                      protected_attribute_names=protected_attributes
                                 )

#### Métricas de equidad del conjunto de datos original

In [98]:
# Metric for the original dataset
original_bias = BinaryLabelDatasetMetric(binary_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Diferencia de resultados medios entre grupos privilegiados y no privilegiados = %f" % original_bias.mean_difference())

Diferencia de resultados medios entre grupos privilegiados y no privilegiados = -0.093750


#### Transformación del conjunto de datos

Existen 4 tipos de transformaciones que se pueden realizar en el conjunto de datos:
- **Reweighing:** Modifica los pesos de las instancias para que el conjunto de datos sea más equitativo.
- **Disparate Impact Remover:** Elimina las características que pueden generar sesgo en el conjunto de datos.
- **LFR (Learning Fair Representations):** Aprende una representación justa del conjunto de datos.
- **Data preprocessing:** Realiza un preprocesamiento de los datos para eliminar el sesgo.

Se ha comprobado que _Data preprocessing_ tarda demasiado como para poder realizarlo y _Dispare impact remover_ necesita el modelo entre medias y no es valido en este caso, por lo que se realizará una transformación con _Reweighing_ y _Disparate Impact Remover_.

In [99]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(binary_dataset)
rw_binary_transformed = RW.transform(binary_dataset)

In [100]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
TR = TR.fit(binary_dataset, maxiter=5000, maxfun=5000)
tr_binary_transformed = TR.transform(binary_dataset)

step: 0, loss: 168199.02849821656, L_x: 1681982.939169717,  L_y: 0.7344608854860396,  L_z: 6.01796835772106e-05
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          220     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.68199D+05    |proj g|=  1.18186D+01
step: 250, loss: 168175.7306718908, L_x: 1681750.240072966,  L_y: 0.7065415742542582,  L_z: 6.150997059308494e-05
step: 500, loss: 168060.08389543605, L_x: 1680594.9612415594,  L_y: 0.5876541985635041,  L_z: 5.854077640549109e-05
step: 750, loss: 167520.7179161489, L_x: 1675207.5848816934,  L_y: -0.040620280092705754,  L_z: 2.4129806197723582e-05
step: 1000, loss: 167490.3856291915, L_x: 1674904.7043270818,  L_y: -0.08485077173628969,  L_z: 2.362752532812505e-05

At iterate    1    f=  1.67490D+05    |proj g|=  6.07539D+01
  ys=-2.466E+02  -gs= 4.803E+02 BFGS update SKIPPED
step: 1250, loss: 163845.26535485886, L_x: 1638560.784508756,  L_y: -10.813096

#### Métricas de equidad del conjunto de datos tras transformar los pesos

In [101]:
rw_transformed_bias = BinaryLabelDatasetMetric(rw_binary_transformed, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
print("Diferencia de resultados medios entre grupos privilegiados y no privilegiados con RW = %f" % rw_transformed_bias.mean_difference())

Diferencia de resultados medios entre grupos privilegiados y no privilegiados con RW = 0.000000


In [102]:
tr_transformed_bias = BinaryLabelDatasetMetric(tr_binary_transformed, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
print("Diferencia de resultados medios entre grupos privilegiados y no privilegiados con TR = %f" % tr_transformed_bias.mean_difference())

Diferencia de resultados medios entre grupos privilegiados y no privilegiados con TR = 0.000000


### Testing

#### Con el dataset con pesos REWEIGHING

In [103]:
# Get the dataset and split into train and test
transformed_train, transformed_test = rw_binary_transformed.split([0.8], shuffle=True)

In [104]:
# Logistic regression classifier and predictions
b_X_train = transformed_train.features
b_y_train = transformed_train.labels.ravel()
b_w_train = transformed_train.instance_weights.ravel()

b_X_test = transformed_test.features
b_y_test = transformed_test.labels.ravel()
b_w_test = transformed_test.instance_weights.ravel() 

model = LogisticRegression(max_iter=1000000)

model.fit(b_X_train, b_y_train, sample_weight=b_w_train)

b_y_pred = model.predict(b_X_test)

accuracy = accuracy_score(b_y_test, b_y_pred, sample_weight=b_w_test) 
print('Accuracy: %.2f' % (accuracy * 100))

Accuracy: 68.63


#### Con el dataset con pesos LFR

In [105]:
# Get the dataset and split into train and test
tr_X = tr_binary_transformed.convert_to_dataframe()[0]
tr_y = tr_X.get('Class')
tr_X = tr_X.drop('Class', axis=1)
b_X_train, b_X_test, b_y_train, b_y_test = train_test_split(tr_X, y, test_size=0.2, random_state=0)

In [106]:
# Logistic regression classifier and predictions

model = LogisticRegression(max_iter=1000000)

model.fit(b_X_train, b_y_train)

b_y_pred = model.predict(b_X_test)

accuracy = accuracy_score(b_y_test, b_y_pred) 
print('Accuracy: %.2f' % (accuracy * 100))


Accuracy: 71.00


#### Con el dataset sin cambios

In [107]:
# read the original csv
df = pd.read_csv('../data/Statlog_preprocesado.csv')

categorical_column = df.select_dtypes(include=['object', 'category']).columns

data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical_column:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))

model = LogisticRegression(max_iter=1000000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 76.00


### Guardar los pesos

Se ha demostrado que el modelo con pesos REWEIGHING es el que mejor se ajusta a los datos, por lo que se guardan los pesos obtenidos para poder utilizarlos en el modelo final.

In [108]:
# save to a csv binary_transformed.instance_weights
df['Weights'] = rw_binary_transformed.instance_weights
df.to_csv('../data/Statlog_weights.csv', index=False)